<div class = "alert alert-block alert-success">

# <span style='color:Blue'>This notebook provides a tutorial for the <span style='color:purple'>nightingale_parallel</span> inference module. We'll use a fully trained OMITTED model provided in Nightingale/Inference/model to overview how to load a model and run inference with the module. 
    
## Make sure you are running the *nightingale_env* kernel in this notebook

In [1]:
from nightingale_parallel import Detector
import time # not required, only used for this notebook
import logging # nightingale_parallel uses python's logging api to log progress
logging.basicConfig(filename='detector_test.log',level=logging.INFO) # check for this file in your directory for logging progress

<div class = "alert alert-block alert-success">

# The <span style='color:purple'>nightingale_parallel</span> module is composed of the "Detector" class and it's "predict" function. You'll use the Detector class to initialize a detector instance, and then run the instance with the "predict" function to perfrom inference on an image. Below is the docstring followed by executable examples. 

## The API usage for instantiating a detector:

```
class Detector():
    
    def __init__(self,gpu_ids,placeholder=(2048,2048,3),allow_growth=False,model='model/omitted_scrdet_Frozen.pb'):
'''
        Assign GPU IDs and Initialize Model Weights
        
        Parameters
        ----------
        gpu_ids : int, [int] or csv string assigning specific GPUs for this process.
                    Current version supports multiple GPUs for parallel processing
                    of large images. 
                    
        placeholder : 3D tuple (rows,cols,channels) sets sliding window size used to
                        process arbitrarily sized imagery and reserves memory on the
                        GPU's. Imagery smaller than placeholder will be zero-padded. 
                        Default is (2048,2048,3). Has been successfully tested up to
                        shape of (4096,4096,3) on a Tesla V100-DGXS-32GB GPU. 
                        
        allow_growth : bool. Whether or not to allow other processes to allocate GPU memory
                        on the GPU's you are using. Default is False. If you are maxing
                        out GPU memeory with very large images (e.g., 4096,4096,3), you
                        will want this set to False. For futher info, see Tensorflow's
                        documentation for tf.ConfigProto gpu_options.allow_growth
                        
        model : string, path to the tensorflow frozen graph .pb file
        '''
```
## The API usage for inferencing the detector instance:
```
    def predict(self,file,clips = None, conf = 0.01,
                class_list = False,
                virtual_mem=False, nms_iou = 0.2, h_overlap = 200,
                w_overlap = 200, max_nms_output = 200):
        

        Parameters
        ----------
        file : str
            Complete path to an image, or path to a directory of images
        
        clips : list[[y0_0,x0_0,y1_0,x1_0],...[y0_n,x0_n,y1_n,x1_n]]
            A list of lists containing pixel values used to clip out a portion or portions of the image for 
            processing. Can be left as  clips = None to process the whole image (this is 
            default). Must be formatted as a list of list, so passing a single clip would be formatted as 
            clips = [[row0,col0,row1,col1]]. Clips are not supported if passing in a directory of 
            images for processing. Pass a single image with clips locations if you want to use clips. If clips are
            passed in, then virtual_mem will automatically be disabled (set to False).
        
        conf : float (0,1] or list of floats where each value corresponds to a specific class confidence
            Output detection if detection confidence score is greater than or equal to this value
            
        class_list : list of strings
            A name for each class
        
        virtual_mem : Bool
            Reduce memory consumption by treating NITF imagery as a virtual array, where smaller portions of the 
            NITF image are read into memory as they are needed for processing. Reduces time up front by 
            preventing the whole NITF being read into memory, but results in longer processing time overall. 
            If clips are passed in, then virtual_mem setting is overwritten and set to False.
            
        nms_iou : float; Non-Max Supression IoU threshold.

        *_overlap : int; h (height/vertical) or w (width/horizontal) pixel overlap for tiled/sliding image 
            processing window(s). Size of processing window is set by the "placeholder" parameter
            when the detector is initialized.

        max_nms_output : int; maximum number of detections to return from nms function. 

        
        Returns
        -------
        pd.DataFrame
            A pandas dataframe containing the following colums:
                * id - int, detection id (index of dataframe)
                * geometry - Shapely formatted polygon in NITF pixel coordinates
                * class - string, object class
                * conf - float in (0,1]
                * image_name - string
                * (class) - list of floats, confidence score for each class, including background
```

# 1) Initialize the detector with default settings using GPU #2 and GPU #0 (assumes you have at least 3 GPU's accessible )

In [2]:
start = time.time()
detector = Detector([2,0])
end = time.time()
print('Elapsed time = ',end-start)

Elapsed time =  79.05209755897522


## Specify an image to run. We'll specify the test image from our "OMITTED_TEST_Data_Nightingale_Format.csv" file. 

In [3]:
file = 'Sample_NITF/omitted.r0.ntf'

# 2a) Run detector on a NITF with some pre-determined NITF clips.
# Note that the clip format is clip=[[y0_0,x0_0,y1_0,x1_0],...[y0_n,x0_n,y1_n,x1_n]]

In [4]:
start = time.time()
clips = [[8000, 11000, 11000, 14000],[10000, 200, 12500, 3000],[15000,15000,17000,17000]]
predictions_clip = detector.predict(file=file,clips=clips,class_list=['class1','class2','class3'])
end = time.time()
print('Elapsed time = ',end-start)

Elapsed time =  13.214110851287842


# 2b) Alternatively, run detector on the whole NITF

In [5]:
# start = time.time()
# predictions_full = detector.predict(file=file,class_list=['class1','class2','class3'])
# end = time.time()
# print('Elapsed time = ',end-start)

# 2c) Or, you can pass in a directory of images

In [6]:
# start = time.time()
# predictions_folder = detector.predict(file='Sample_NITF',class_list=['class1','class2','class3'])
# end = time.time()
# print('Elapsed time = ',end-start)

# 3) Verify the output format

In [1]:
print(type(predictions_clip))
predictions_clip.head()

# 4) Save the detections dataframe to a csv

In [8]:
predictions_clip.to_csv('Test&Evaluate/my_results.csv',index=False)

<div class = "alert alert-block alert-success">
    
# From here, you can now open the <span style='color:purple'>Test&Evaluate</span> folder to evaluate your results text file. The next cells are just an example for plotting the results on your NITF. 

# 5) Plot the whole NITF with prediction results from the clip run

In [9]:
## This cell imports tools for reading and plotting
from matplotlib import pyplot as plt
from matplotlib.patches import Patch
from matplotlib import cm, colors
from osgeo.gdal import Open as gdalOpen
import cv2
import numpy as np

In [11]:
## This cell reads and adjusts an image

##################
#### READ PNG ####
##################
#img = cv2.imread(file)
######################
######################

##################
### READ NITF ####
##################
my_nitf = gdalOpen(file)
text = 'Reading Image...'
print(text,end="\r")
img = my_nitf.GetRasterBand(1).ReadAsArray()
#################################################
#### optionally adjust nitf visibility ##########
from nightingale_parallel import adjust_image ########
bits = int(my_nitf.GetMetadata()['NITF_ABPP'])###
img = adjust_image(img,bit_depth_in=bits) #######
#################################################
#################################################
print(' '*len(text),end="\r")
print('Done')
##########################
##########################

Done            


In [2]:
## This cell plots everything
fig1,ax = plt.subplots(figsize=(100,100))
ax.imshow(img,cmap='gray');
classes = np.array(sorted(predictions_clip['class'].unique()))
colors = cm.get_cmap('rainbow',len(classes))

# plot the detections
for index, row in predictions_clip[predictions_clip['conf'] > 0.8].iterrows():
    cli = np.where(classes == row['class'])[0][0]
    ax.plot(*row['geometry'].exterior.xy,c=colors(cli))
    
# plot the clips    
for clip in clips:
    xs = [clip[1],clip[1],clip[3],clip[3],clip[1]]
    ys = [clip[0],clip[2],clip[2],clip[0],clip[0]]
    ax.plot(xs,ys,'g',linewidth=4)

legend_elements = []
for c in classes:
    cli = np.where(classes == c)[0][0]
    legend_elements.append(Patch(color=colors(cli),label=c))
ax.legend(handles=legend_elements,loc='upper left',fontsize=60);

In [ ]:
# image output omitted